In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os # -> for directory separator

In [2]:
from laser_annotation import MetadataGenerator, SegmentPointsProvider, SegmentAnnotator

## Load the Versuchstabelle + delays

In [3]:
path_meta = '/Volumes/Sandisk_SD/Work/IZFP/Laser/2023_iHub/00_Versuchsplan_iHub.xlsx'
versuche = pd.read_excel(
    path_meta,
    sheet_name=1,
    usecols="C:G, I, R,S",
    nrows=71,
    decimal=','
)
versuche

,HV-Data,QASS,SoundBook,DEWETRON,Probe,Spalt [mm],tstart [s],tend [s]
0,01_0_1_Spalt_pneu,45,iHub2023_001,01_0_1_Spalt_pneu,standard,0.1,0.2520,1.0606
1,02_0_1_Spalt_pneu,46,iHub2023_002,02_0_1_Spalt_pneu,standard,0.1,0.2504,1.0566
2,03_0_1_Spalt_pneu,47,iHub2023_003,03_0_1_Spalt_pneu,standard,0.1,0.2501,1.0581
3,04_0_1_Spalt_pneu,48,iHub2023_004,04_0_1_Spalt_pneu,standard,0.1,0.2514,1.0594
4,05_0_1_Spalt_pneu,49,iHub2023_005,05_0_1_Spalt_pneu,standard,0.1,0.2513,1.0583
...,...,...,...,...,...,...,...,...
66,67_0_2_Spalt_pneu_CJ_X2,130,iHub2023_075,67_0_2_Spalt_pneu_CJ_X2,standard,0.2,0.2489,1.0561
67,68_0_2_Spalt_pneu_CJ_X2,131,iHub2023_076,68_0_2_Spalt_pneu_CJ_X2,standard,0.2,0.2500,1.0564
68,69_0_2_Spalt_pneu_CJ_X2,132,iHub2023_077,69_0_2_Spalt_pneu_CJ_X2,standard,0.2,0.2498,1.0557
69,70_0_2_Spalt_pneu_CJ_X2,133,iHub2023_078,70_0_2_Spalt_pneu_CJ_X2,standard,0.2,0.2494,1.0563


In [4]:
path_delay = '/Volumes/Sandisk_SD/Work/IZFP/Laser/2023_iHub/04_delays'
delay_df = pd.read_csv(f'{path_delay}/delays_QASS.csv') 
delay_df

,dataID,DEWETRON,QASS,delay[s]
0,1,01_0_1_Spalt_pneu,45,0.001382
1,2,02_0_1_Spalt_pneu,46,0.000860
2,3,03_0_1_Spalt_pneu,47,0.008059
3,4,04_0_1_Spalt_pneu,48,0.002406
4,5,05_0_1_Spalt_pneu,49,0.005622
...,...,...,...,...
66,67,67_0_2_Spalt_pneu_CJ_X2,130,0.000451
67,68,68_0_2_Spalt_pneu_CJ_X2,131,0.007772
68,69,69_0_2_Spalt_pneu_CJ_X2,132,0.000451
69,70,70_0_2_Spalt_pneu_CJ_X2,133,0.008878


## Generate the metadata -> v.23/11/22: adjusted to iHub 2023

In [21]:
T_seg = 100.0*10**-3 #[s]

metagen = MetadataGenerator()
metagen.set_constants(
    versuche_df = versuche, 
    delay_df = delay_df,
    T_seg = T_seg, #[s], segment duration
    t_buffer = 5.0*10**-3, # buffer at the beginning and the end of each patch
    incl_transition = False, # True for including the transitions
    binary = False,  # True for binary classification
    steps = 10*10**-3, #[s], step size between two consecutive segments in a patch
    patches=[1, 2, 3, 4, 5] # specify which patches to be included 
)

In [26]:
metadata = {} # base

for trialNo in list(versuche['DEWETRON'][:22]):
    metagen.trialNo = trialNo
    metadata.update(metagen.generate_trial_metadata(len(metadata)))

## Analyze the labels

In [27]:
# Convert into pandas dataframe
metadf = pd.DataFrame.from_dict(metadata).T
metadf

,dataID,trial,QASS,T_seg[ms],segment,t[s],patch,completeness,gap[mm],classID,class
0,0,01_0_1_Spalt_pneu,45,100.0,0,0.255618,P1,1.0,0.0,0,noGap
1,1,01_0_1_Spalt_pneu,45,100.0,1,0.265618,P1,1.0,0.0,0,noGap
2,2,01_0_1_Spalt_pneu,45,100.0,2,0.275618,P1,1.0,0.0,0,noGap
3,3,01_0_1_Spalt_pneu,45,100.0,3,0.285618,P1,1.0,0.0,0,noGap
4,4,01_0_1_Spalt_pneu,45,100.0,4,0.295618,P1,1.0,0.0,0,noGap
...,...,...,...,...,...,...,...,...,...,...,...
655,655,22_0_3_Spalt_pneu,66,100.0,25,0.907779,P5,1.0,0.0,0,noGap
656,656,22_0_3_Spalt_pneu,66,100.0,26,0.917779,P5,1.0,0.0,0,noGap
657,657,22_0_3_Spalt_pneu,66,100.0,27,0.927779,P5,1.0,0.0,0,noGap
658,658,22_0_3_Spalt_pneu,66,100.0,28,0.937779,P5,1.0,0.0,0,noGap


In [28]:
print('Number of samples in each class')
metadf.value_counts(subset=['classID'])

Number of samples in each class


classID
0          396
1          120
2           84
3           60
Name: count, dtype: int64

## Save the metadata

In [29]:
path2save = '/Volumes/Sandisk_SD/Work/IZFP/Laser/2023_iHub/05_Evaluation'
# Save
fname = f'{path2save}{os.sep}metadata.csv'
metadf.to_csv(fname, index=False) # indexineg is turned off, as metadf is already indexed